In [1]:
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5 import *
from deepface import DeepFace 
import sys
import os
import cv2
import pandas as pd

In [2]:
# trying to see if global variables will work

# need to watch movie videos/review about how the check box class works. create signals/functions to identify when the state is changed

#these don't change
single_cb_checked = False
batch_cb_checked = False
age_cb_checked = False
gender_cb_checked = False
race_cb_checked = False
emotion_cb_checked = False

threshold = 50

data = pd.DataFrame()

class QComboBox(QtWidgets.QComboBox):
    def __init__(self, parent=None):
        super(QIComboBox, self).__init__(parent)
        
class Wizard(QtWidgets.QWizard):
    def __init__(self, parent=None):
        super(Wizard, self).__init__(parent)

        #add page 1,2
        self.page1 = Page1()
        self.addPage(self.page1)
        
        self.page2 = Page2()
        self.addPage(self.page2)
        
        #set ids for all potential pages
        self.page3singleid = self.addPage(Page3Single())
        self.page4single = Page4Single()
        self.page4single.setFinalPage(True)
        self.page4singleid = self.addPage(self.page4single)
        
        self.page3batchid = self.addPage(Page3Batch())
        self.page4batch = Page4Batch()
        self.page4batch.setFinalPage(True)
        self.page4batchid = self.addPage(self.page4batch)
        
        #if else doesn't seem to make a difference
        # currently using nextId() function, but isn't working
        
        #shows user all pages, not just single/batch based on what they check
        
        self.setWindowTitle("DCiFR")
        self.setGeometry(0, 0, 800, 600)
        
        def nextId(self):
            if self.currentPage() == self.page2:
                if self.page2.batch_cb.isChecked():
                    return self.page3batchid
                else:
                    return self.page3singleid
            if self.currentPage() == self.page3singleid:
                return self.page4single
            if self.currentPage() == self.page3batchid:
                return self.page4batch
        return nextId(self)
        
class Page1(QtWidgets.QWizardPage):
    def race_cb_changed(self, state):
            if state == Qt.Checked:
                race_cb_checked = True
        
    def gender_cb_changed(self, state):
            if state == Qt.Checked:
                gender_cb_checked = True
        
    def age_cb_changed(self, state):
            if state == Qt.Checked:
                age_cb_checked = True
    def emotion_cb_changed(self, state):
            if state == Qt.Checked:
                emotion_cb_checked = True
    def __init__(self, parent=None):
        super(Page1, self).__init__(parent)
        
        self.title_label = QLabel('Welcome to DCiFR!', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()

        self.title_label = QLabel('Attributes', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 15))
        self.title_label.adjustSize()

        #hover info 
        info = QLabel('Check the boxes that apply. Hover for more info!', self)
        info.move(50, 125)
        info.setFont(QFont('Arial', 10))
        info.adjustSize()
        myFont=QtGui.QFont()
        myFont.setItalic(True)
        info.setFont(myFont)
        info.adjustSize()
        
        #Check boxes
        age_cb = QCheckBox('Age', self)
        age_cb.move(50, 150)
        race_cb = QCheckBox('Race', self)
        race_cb.move(50, 200)
        age_cb.adjustSize()
        race_cb.adjustSize()
        gender_cb = QCheckBox('Gender', self)
        gender_cb.move(50, 250)
        emotion_cb = QCheckBox('Emotion', self)
        emotion_cb.move(50, 300)
        gender_cb.adjustSize()
        emotion_cb.adjustSize()
        
        # connect to functions
        age_cb.stateChanged.connect(self.age_cb_changed)
        race_cb.stateChanged.connect(self.race_cb_changed)
        emotion_cb.stateChanged.connect(self.emotion_cb_changed)
        gender_cb.stateChanged.connect(self.gender_cb_changed)
        
        #Adding slider for race attribute
        self.sld = QSlider(Qt.Horizontal, self)
        self.sld.setRange(0, 100)
        self.sld.move(500, 240)
        self.sld.setTickInterval(10)
        label = QLabel('Please select your threshold value \n(only applicable if you select the race attribute)', self)
        label.move(300, 150)
        minlabel = QLabel('0', self)
        minlabel.move(500, 260)
        maxlabel = QLabel('100', self)
        maxlabel.move(570, 260)
        q1label = QLabel('25', self)
        q1label.move(510, 260)
        medlabel = QLabel('50', self)
        medlabel.move(530, 260)
        q3label = QLabel('75', self)
        q3label.move(550, 260)
        minlabel.setFont(QFont('Arial', 5))
        maxlabel.setFont(QFont('Arial', 5))
        q1label.setFont(QFont('Arial', 5))
        medlabel.setFont(QFont('Arial', 5))
        q3label.setFont(QFont('Arial', 5))
        minlabel.adjustSize()
        maxlabel.adjustSize()
        q1label.adjustSize()
        medlabel.adjustSize()
        q3label.adjustSize()
        
        # tool tip
        self.sld.setToolTip('This is a slider for the threshold of the race attribute.')
            # need to connect slider value to changing threshold value
        #if (self.sld.valueChanged()):
        #    threshold = self.sld.value()
        #Hovers
        age_cb.setToolTip('Check this box if you would like to analyze the age of the subject in your image(s)')
        race_cb.setToolTip('Check this box if you would like to analyze the race of the subject in your image(s)')
        gender_cb.setToolTip('Check this box if you would like to analyze the gender of the subject in your image(s)')
        emotion_cb.setToolTip('Check this box if you would like to analyze the emotion of the subject in your image(s)')
        
        age_cb_checked = age_cb.isChecked()
        gender_cb_checked = gender_cb.isChecked()
        emotion_cb_checked = emotion_cb.isChecked()
        race_cb_checked = race_cb.isChecked()
        
        #print(age_cb_checked, gender_cb_checked, emotion_cb_checked, race_cb_checked)
        
class Page2(QtWidgets.QWizardPage):
    def single_cb_changed(self, state):
            if state == Qt.Checked:
                single_cb_checked = True
                
    def batch_cb_changed(self, state):
            if state == Qt.Checked:
                batch_cb_checked = True
    def __init__(self, parent=None):
        super(Page2, self).__init__(parent)
        self.title_label = QLabel('DCiFR', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()
        
        self.title_label = QLabel('Single or Batch Mode', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 15))
        self.title_label.adjustSize()
        #hover info 
        info = QLabel('Check the box that applies. Hover for more info!', self)
        info.move(50, 125)
        info.setFont(QFont('Arial', 10))
        info.adjustSize()
        myFont=QtGui.QFont()
        myFont.setItalic(True)
        info.setFont(myFont)
        info.adjustSize()
        
        #Check boxes - hbox allows these to be exclusive
        hbox = QHBoxLayout()
    
        self.single_cb = QCheckBox('Single Image', self)
        self.single_cb.move(50, 150)
        self.batch_cb = QCheckBox('Batch Mode', self)
        self.batch_cb.move(50, 200)
        self.single_cb.adjustSize()
        self.batch_cb.adjustSize()
        
        self.single_cb.stateChanged.connect(self.single_cb_changed)
        self.batch_cb.stateChanged.connect(self.batch_cb_changed)
        
        group = QButtonGroup(self)
        group.addButton(self.single_cb)
        group.addButton(self.batch_cb)
        
        hbox.addWidget(self.single_cb)
        hbox.addWidget(self.batch_cb)
        
        #Hovers
        self.single_cb.setToolTip('Check this box if you would like to analyze demograhpics for a single image')
        self.batch_cb.setToolTip('Check this box if you would like to analyze demograhpics for more than one image')

class Page3Single(QtWidgets.QWizardPage):
    #upload and analyze a single image
    
    # DeepFace integration
    # Checks for one face, and if there is ONLY one face, will perform analysis
    
    #TO-DO: add if/else conditions based on checkbox statuses
        # need way to reference values from page 1
    
    def get_image_file(self):
            dialog = QFileDialog()
            file_name = dialog.getOpenFileName(self, 'Open image')
            file = os.path.join(file_name[0])
            print(file)
            self.detect_face_show(file)
            
    # this isn't doing anything        
    def detect_face_show(self, fpath):
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        img = cv2.imread(fpath)
        if img is None:
            return(0)
        else:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            faces = face_cascade.detectMultiScale(gray, 1.1, 4)
            face_num = len(faces)
            data['filepath'] = pd.Series(fpath)
            filename = fpath.split("\\")
            data['id'] = pd.Series(filename)
            data['faces'] = face_num
            if (face_num == 1):
                result = DeepFace.analyze(faces, ['race'], enforce_detection=False)
                print(result)
                print("done!")
            else:
                return(0)
            
    def __init__(self, parent=None):
        super(Page3Single, self).__init__(parent)
        self.title_label = QLabel('DCiFR', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()
        self.title_label = QLabel('Upload Your Image Below', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 10))
        self.title_label.adjustSize()
        
        self.button1 = QPushButton("Select An Image to Upload Here", self)  
        self.button1.clicked.connect(self.get_image_file) 
        self.button1.move(50, 150)
        
class Page3Batch(QtWidgets.QWizardPage):
    #upload and analyze multiple images
    
    def get_image_files(self):
        
        #glob
        dialog = QFileDialog()
        dialog.setOption(dialog.DontUseNativeDialog, True)
        file_name = dialog.getExistingDirectory(self, "Select A Folder")
        file = os.path.join(file_name)
        print(file)
        
    def __init__(self, parent=None):
        super(Page3Batch, self).__init__(parent)
        self.title_label = QLabel('DCiFR', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()
        self.title_label = QLabel('Upload Your Images Below', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 10))
        self.title_label.adjustSize()
        
        self.button1 = QPushButton("Select Your Folder of Images to Upload Here", self)   
        self.button1.clicked.connect(self.get_image_files)
        self.button1.move(50, 150)
        
class Page4Single(QtWidgets.QWizardPage):
    def __init__(self, parent=None):
        super(Page4Single, self).__init__(parent)
        
        self.title_label = QLabel('DCiFR', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()

        self.title_label = QLabel('Results', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 15))
        self.title_label.adjustSize()
        self.title_label = QLabel('Here are the results for the IMAGE you uploaded:', self)
        self.title_label.move(100, 125)
        self.title_label.setFont(QFont('Arial', 10))
        self.title_label.adjustSize()
        
class Page4Batch(QtWidgets.QWizardPage):
    def __init__(self, parent=None):
        super(Page4Batch, self).__init__(parent)
        
        self.title_label = QLabel('DCiFR', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()

        self.title_label = QLabel('Results', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 15))
        self.title_label.adjustSize()
        self.title_label = QLabel('Here are the results for the IMAGES you uploaded:', self)
        self.title_label.move(100, 125)
        self.title_label.setFont(QFont('Arial', 10))
        self.title_label.adjustSize()
        
if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    wizard = Wizard()
    wizard.show()
    app.exec_()   
    print(age_cb_checked, gender_cb_checked, emotion_cb_checked, race_cb_checked, single_cb_checked, batch_cb_checked)

    
#TO-DO:

# deepface connection isn't producing any results?
# work on page 3 single/batch pipeline

C:/Users/melin/OneDrive/Pictures/IMG_7600.jpg
False False False False False False
